In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

sdkfsdkjf
# IV is a data structure containing all data collected in a given execution of lockin_iv.
class IV_ami:
    def __init__(self, start_field, final_field, ramp_rate, delay):
        field = np.linspace(start_field,final_field,round((final_field - start_field)/ramp_rate))
        self.field = field # array of field values
        self.mag_field = np.zeros(len(field)) # array of mag field readings from lockin
        self.ac_voltagexx = np.zeros(len(field)) # array of ac voltage readings from lockin
        self.ac_voltagexy = np.zeros(len(field)) # array of ac voltage readings from lockin
        self.ac_current = np.zeros(len(field)) # array of ac voltage readings from lockin
        self.phase = np.zeros(len(field)) # array of phase readings from lockin
        self.ramp_rate = ramp_rate # ramp rate
        self.delay = delay # delay between reading values

# lockin_iv is a function that sequentially sets a keithley to a specified array of voltages. at each
# voltage, the dc current on the keithley and the ac voltage and phase read by the lockin are recorded.
# at the end of execution, the data will be saved to an IV object (defined above). this IV object will
# also be written to a file for later retrieval.

# input parameters:
#    k2400 is a keithley object, assumed initialized
#    srsxx is a srs lockin object, assumed initialized, for the xx voltage
#    srsxy is a srs lockin object, assumed initialized, for the xy voltage
#    srsxy is a srs lockin object, assumed initialized, for the sample current
#    dc_voltage is an array of dc voltages for the keithley
#    delay is the time to delay before acquiring data at each dc voltage
#    step is step in the experiment

def lockin_iv_ami(ami,srsxx,srsxy,srscurrent,start_field,final_field,ramp_rate, delay, step):
    %matplotlib inline
    
    # create new IV object to store data in
    iv = IV_ami(start_field,final_field,ramp_rate, delay)
    
    # make a filename to save the data, for later use
    full_filename = makefilename(None, step)
    print('step is' + str(step))

    # define a figure with two panels to show the data being acquired
    fig = plt.figure(figsize=(8, 3)) 
    
    fig.suptitle(full_filename)
    ax1=plt.subplot(2,2,1)
    ax1.set_xlabel('mag field (T)')
    ax1.set_ylabel('ac voltage xx (A)')
    plt.plot(iv.mag_field,iv.ac_voltagexx)
    
    ax2=plt.subplot(2,2,2)
    ax1.set_xlabel('mag field (T)')
    ax2.set_ylabel('input field (T)')
    plt.plot(iv.mag_field,iv.field)
    plt.tight_layout()
    
    ax2=plt.subplot(2,2,3)
    ax1.set_xlabel('mag field (T)')
    ax2.set_ylabel('ac current xy (A)')
    plt.plot(iv.mag_field,iv.ac_voltagexy)
    plt.tight_layout()
    
    ax2=plt.subplot(2,2,4)
    ax1.set_xlabel('mag field (T)')
    ax2.set_ylabel('sample current (A)')
    plt.plot(iv.mag_field,iv.ac_current)
    plt.tight_layout()
    


    
    
    try:
        # the for loop below is the main acquisition loop. the reason the for loop lives in a try-except
        # is that if the user cancels while the sweep is running, which throws an exception, the program 
        # catches it and continues to execute below the except statement. this lets the program shut down
        # nicely and save what data has already been collected.
        
        rampami(final_field, ramp_rate)
        
        for i in range(len(iv.field)):
            
            # read data and store it
            iv.mag_field[i]=readami(ami)
            iv.ac_voltagexx[i],iv.phase[i]=readsrs(srsxx)
            iv.ac_voltagexy[i],iv.phase[i]=readsrs(srsxy)
            iv.ac_current[i],iv.phase[i]=readsrs(srscurrent) #TODO--is this still readsrs?
            
            # update plots
            ax1.lines[0].set_xdata(iv.mag_field[0:(i+1)])
            ax1.lines[0].set_ydata(iv.ac_voltagexx[0:(i+1)])
            ax1.relim()
            ax1.autoscale()

            ax2.lines[0].set_xdata(iv.mag_field[0:(i+1)])
            ax2.lines[0].set_ydata(iv.field[0:(i+1)])
            ax2.relim()
            ax2.autoscale()
            plt.tight_layout()
            
            ax3.lines[0].set_xdata(iv.mag_field[0:(i+1)])
            ax3.lines[0].set_ydata(iv.ac_voltagexy[0:(i+1)])
            ax3.relim()
            ax3.autoscale()
            plt.tight_layout()

            ax4.lines[0].set_xdata(iv.mag_field[0:(i+1)])
            ax4.lines[0].set_ydata(iv.ac_current[0:(i+1)])
            ax4.relim()
            ax4.autoscale()
            plt.tight_layout()


            display.clear_output(wait=True)
            display.display(plt.gcf())
            time.sleep(iv.delay)

    except: 
        ; # do nothing if there was an exception. just proceed to what follows the try-except statement
        
    display.clear_output(wait=True)
    
    # save data as a .pkl file for use with python
    savedata(full_filename,iv)
    
    # also save data as a text file for easy opening with other software
    np.savetxt(full_filename+'.txt',np.transpose(np.array([iv.field,iv.mag_field,iv.ac_voltagexx,iv.ac_voltagexy,iv.ac_current,iv.phase])),newline='\r\n')
    
    return iv

